In [ ]:
import approval_que_app as aqa

In [ ]:
working_session = aqa.OracleConnect()
working_session_backup = aqa.TableBackUp(working_session)

# Create session backups
_result = working_session_backup.perform_session_backup()
print(_result)
# Display created file names
print(working_session_backup.session_bk_FORAQUS_file_name)
print(working_session_backup.session_bk_FORAQRC_file_name)
print(working_session_backup.session_bk_FTVAPPQ_file_name)


In [ ]:
working_session.engine_connection.close()

In [ ]:
#returns who in the queues which they are in

_select = f"""select distinct FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR,
listagg(distinct FORAQRC_ORGN_CODE,', '), listagg(distinct FTVORGN_TITLE,', ')"""
_from = f"""from FORAQUS, FORAQRC, FTVORGN"""
_where = f"""where FORAQUS_QUEUE_ID like '%B%'
and FORAQUS_NCHG_DATE is null
and FORAQUS_TERM_DATE is null
and replace(FORAQUS_QUEUE_ID, 'B', '') = replace(FORAQRC_QUEUE_ID, 'A', '')
and FTVORGN_ORGN_CODE = FORAQRC_ORGN_CODE"""
_by = f"""group by FORAQUS_QUEUE_ID, FORAQUS_USER_ID_APPR
order by FORAQUS_USER_ID_APPR, FORAQUS_QUEUE_ID """

x = working_session.db_query(cur,_select,_from,_where,_by)
for y in x:
    print(y)

In [ ]:
approver_un = 'SMILLER66'
approver_un = 'SMILLER66'
approver_un = 'SMILLER66'
approver_un = 'BJESTER1'
cur = working_session
_select = f"""foraqus.FORAQUS_QUEUE_ID,foraqus.FORAQUS_USER_ID_APPR,
(select count(FORAQUS_USER_ID_APPR)
from foraqus fa
where fa.FORAQUS_USER_ID_APPR != '{approver_un}'
      and fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers"""
_from = f"""foraqus"""
_where = f"""foraqus.FORAQUS_USER_ID_APPR = '{approver_un}'
and foraqus.FORAQUS_TERM_DATE is null
and foraqus.FORAQUS_NCHG_DATE is null"""
_order_by = """"""
_group_by = """"""
_max_records_returned = """"""

x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
for y in x:
    print(y)

In [ ]:
# search for empty queues
def search_for_mt_queues():
    mt_list = []
    _select = f"""foraqus.FORAQUS_QUEUE_ID,
    (select count(FORAQUS_USER_ID_APPR)
    from foraqus fa
    where fa.FORAQUS_QUEUE_ID = foraqus.FORAQUS_QUEUE_ID
                                and fa.FORAQUS_TERM_DATE is null
                                and fa.FORAQUS_NCHG_DATE is null) as OtherApprovers"""
    _from = f"""foraqus"""
    _where = f"""foraqus.FORAQUS_TERM_DATE is null
    and foraqus.FORAQUS_NCHG_DATE is null"""
    _order_by = """2"""
    _group_by = """"""
    _max_records_returned = """"""
    x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
    for y in x:
        if y[1] == 0:
            print('alert', y)
            mt_list.append(y)
        print(y)
    return mt_list
search_for_mt_queues()

In [ ]:
def orgn_to_queue(orgns):
    _select = f"""distinct FORAQRC_ORGN_CODE, SUBSTR( FORAQRC_QUEUE_ID, 1, 3 )"""
    _from = f"""FORAQRC"""
    _where = f"""FORAQRC_ORGN_CODE in ({orgns}) and SUBSTR( FORAQRC_QUEUE_ID, 1, 1 ) != 'B'"""
    _order_by = f""""""
    _group_by = f""""""
    _max_records_returned = f""""""

    x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
    return x


list1 = orgn_to_queue("'13205', '13206', '62452'")
print(list1)
list2 = orgn_to_queue("'1001','1002','3001','1400'")
print(list2)

In [ ]:
def queue_to_orgn(orgns):
    _select = f"""distinct FORAQRC_QUEUE_ID, FORAQRC_ORGN_CODE"""
    _from = f"""FORAQRC"""
    _where = f"""FORAQRC_QUEUE_ID in ({orgns})"""
    _order_by = f""""""
    _group_by = f""""""
    _max_records_returned = f""""""

    x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
    return x

In [ ]:
list1 = queue_to_orgn("'024A','024B','B024'")
print(list1)

In [ ]:

def get_orgn_pred_info(orgn):
    _select = f"""FTVORGN.FTVORGN_ORGN_CODE,FTVORGN.FTVORGN_TITLE,FTVORGN.FTVORGN_FMGR_CODE_PIDM FMGR_PIDM,
case when FTVORGN.FTVORGN_FMGR_CODE_PIDM is not null then baninst1.Gokname.f_get_name(FTVORGN.FTVORGN_FMGR_CODE_PIDM, 'PREF_FMIL') else 'None Assigned' end,
FTVORGN.FTVORGN_ORGN_CODE_PRED"""
    _from = f"""FIMSMGR.FTVORGN FTVORGN"""
    _where = f"""FTVORGN.FTVORGN_TERM_DATE is null
and FTVORGN.FTVORGN_ORGN_CODE = (select FTVORGN_ORGN_CODE_PRED
from ftvorgn
where FTVORGN_TERM_DATE is null
and FTVORGN.FTVORGN_STATUS_IND = 'A'
and (FTVORGN.FTVORGN_NCHG_DATE > sysdate or FTVORGN.FTVORGN_NCHG_DATE is null)
                                and FTVORGN_ORGN_CODE = {orgn})
and FTVORGN.FTVORGN_STATUS_IND = 'A'
and (FTVORGN.FTVORGN_NCHG_DATE > sysdate or FTVORGN.FTVORGN_NCHG_DATE is null)"""
    _order_by = f""""""
    _group_by = f""""""
    _max_records_returned = f""""""

    x = working_session.db_query(_select,_from,_where,_order_by,_group_by,_max_records_returned)
    return x




In [ ]:
orgns_to_inv = '11050', '11200', '11800', '12930'
orgn_preds = []
for x in orgns_to_inv:
    a = get_orgn_pred_info(x)
    orgn_preds.append(a)


In [ ]:
print(orgn_preds)

for x in orgn_preds:
    print(x[0][0])

In [ ]:
get_orgn_pred_info(10)